In [106]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_core.tools import tool

In [107]:
model = ChatOllama(model="qwen2.5-coder:7b", temperature=0.1,base_url="http://localhost:11434")

In [108]:
memory = MemorySaver()

In [109]:
search =DuckDuckGoSearchResults()
tools = [search]

In [110]:
model_with_tools = model.bind_tools(tools)

In [101]:
response = model_with_tools.invoke([{"role": "user", "content": "Hi"}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")



Message content: Hello! How can I assist you today?

Tool calls: []


In [ ]:
response = model_with_tools.invoke([{"role": "user", "content": "Search for the weather in Dhaka"}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: {"name": "duckduckgo_results_json", "arguments": {"query": "weather in Dhaka"}}

Tool calls: []


In [103]:
agent_executor = create_react_agent(model, tools)

In [104]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I assist you today?


In [105]:
input_message = {"role": "user", "content": "Search for the weather in Dhaka"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Dhaka
================================== Ai Message ==================================

{
  "name": "duckduckgo_results_json",
  "arguments": {
    "query": "weather in Dhaka"
  }
}


In [111]:
@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]

In [112]:
from langchain_core.messages import ToolMessage
from langgraph.func import entrypoint, task

In [113]:
tools_by_name = {tool.name: tool for tool in tools}
tools_by_name

{'get_weather': StructuredTool(name='get_weather', description='Call to get the weather from a specific location.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x0000028F5DAD6700>)}

In [114]:
@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


In [115]:
@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])

In [92]:
config = {"configurable": {"thread_id": "1"}}

In [95]:
# Run the agent
input_message = {
    "role": "user",
    "content": "Hi, I'm Rashel and I live in Dhaka.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()



================================ Human Message =================================

Hi, I'm Rashel and I live in Dhaka.
================================== Ai Message ==================================

Hello Rashel! How can I assist you today? If you have any questions or need information, feel free to let me know!


In [98]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="update"
):
    step["messages"][-1].pretty_print()